In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math

cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("Model/keras_model.h5", "Model/labels.txt")

offset = 20
imgSize = 300
folder = "Data/C"
counter = 0
labels = ["Help", "Hello", "Good", "No","Thanks"]

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]
        imgCropShape = imgCrop.shape
        aspectRatio = h / w
        
        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
            print(prediction, index)
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
        
        cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
        cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)
        
        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)
    
    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


2024-04-12 11:01:04.528 Python[1398:42803] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


1/1 [==============================] - 0s 17ms/step
[0.0026314363, 4.6587196e-05, 6.671558e-05, 0.0007546651, 0.9965006] 4
1/1 [==============================] - 0s 18ms/step
[0.019189429, 4.7401787e-05, 6.922698e-05, 9.805136e-05, 0.98059595] 4
1/1 [==============================] - 0s 17ms/step
[0.0062448727, 1.346385e-05, 5.832628e-05, 4.3540083e-05, 0.99363977] 4
1/1 [==============================] - 0s 17ms/step
[0.00714087, 0.00085411087, 6.829874e-05, 0.00018212196, 0.99175453] 4
1/1 [==============================] - 0s 16ms/step
[6.5786444e-06, 1.1528556e-06, 8.653743e-06, 9.620787e-06, 0.999974] 4
1/1 [==============================] - 0s 16ms/step
[0.00028930584, 0.017277142, 0.0034298683, 0.04200696, 0.9369967] 4
1/1 [==============================] - 0s 17ms/step
[0.0003227443, 0.3230716, 0.0025694564, 0.014820053, 0.6592161] 4
1/1 [==============================] - 0s 17ms/step
[0.00025285376, 0.3809927, 0.0027470335, 0.051266827, 0.56474054] 4
1/1 [=====================

1/1 [==============================] - 0s 16ms/step
[0.00026693763, 0.000104943414, 0.0015168213, 0.0009615186, 0.99714977] 4
1/1 [==============================] - 0s 16ms/step
[0.014878908, 0.0012329762, 0.010721512, 0.0024437304, 0.9707228] 4
1/1 [==============================] - 0s 16ms/step
[0.0012548079, 0.0009270516, 0.00019415302, 0.0004194535, 0.9972045] 4
1/1 [==============================] - 0s 16ms/step
[0.0038129268, 0.6270482, 0.0015858781, 0.14856161, 0.2189914] 1
1/1 [==============================] - 0s 17ms/step
[0.0029782662, 0.76065826, 0.0007098791, 0.011457835, 0.22419569] 1
1/1 [==============================] - 0s 17ms/step
[0.0011076735, 0.96999043, 5.0910377e-05, 0.0047876257, 0.024063474] 1
1/1 [==============================] - 0s 16ms/step
[0.0022878929, 0.75318277, 0.00023451829, 0.029027808, 0.2152669] 1
1/1 [==============================] - 0s 16ms/step
[0.0022178846, 0.6175833, 0.00046227395, 0.008843616, 0.37089288] 1
1/1 [==========================

1/1 [==============================] - 0s 16ms/step
[0.0071185683, 6.840881e-05, 0.1464732, 0.8460067, 0.00033311106] 3
1/1 [==============================] - 0s 16ms/step
[0.006146185, 6.7552384e-05, 0.2161006, 0.77585816, 0.0018275109] 3
1/1 [==============================] - 0s 16ms/step
[0.0030625202, 2.2041493e-05, 0.05913894, 0.93720627, 0.00057029125] 3
1/1 [==============================] - 0s 16ms/step
[0.010605766, 5.2099407e-05, 0.10369486, 0.88250464, 0.003142611] 3
1/1 [==============================] - 0s 16ms/step
[0.024038525, 0.00010382102, 0.098867096, 0.8762181, 0.0007725215] 3
1/1 [==============================] - 0s 16ms/step
[0.004280166, 5.7977613e-05, 1.1938786e-06, 4.30484e-06, 0.9956564] 4
1/1 [==============================] - 0s 15ms/step
[0.49867558, 0.00042250534, 0.00013772152, 0.0003255204, 0.5004387] 4
1/1 [==============================] - 0s 16ms/step
[0.50847924, 0.00022016489, 3.9012746e-05, 0.00029974338, 0.49096182] 0
1/1 [=======================

1/1 [==============================] - 0s 15ms/step
[0.025719294, 0.0015292084, 0.67845666, 0.010973292, 0.28332147] 2
1/1 [==============================] - 0s 16ms/step
[0.007948248, 0.0026651036, 0.42578033, 0.08263248, 0.48097384] 4
1/1 [==============================] - 0s 16ms/step
[0.021011679, 0.028697876, 0.72501147, 0.16174722, 0.06353176] 2
1/1 [==============================] - 0s 16ms/step
[0.00035307038, 0.00011112067, 0.0018022832, 0.0034422595, 0.9942913] 4
1/1 [==============================] - 0s 16ms/step
[0.22497165, 0.014472118, 0.4969306, 0.00036996143, 0.26325575] 2
1/1 [==============================] - 0s 16ms/step
[0.13996135, 0.0016662222, 0.31058642, 0.0001956293, 0.5475904] 4
1/1 [==============================] - 0s 16ms/step
[0.049534693, 0.0026633663, 0.60233575, 0.00026991352, 0.34519628] 2
1/1 [==============================] - 0s 16ms/step
[0.04835202, 0.0048013004, 0.56040245, 0.00019899583, 0.38624525] 2
1/1 [==============================] - 0s 16m